In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
import datetime
import random
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [305]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, 'L' code_mode_exercice  from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [272]:
def get_pa(year, specialite):
    sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
--and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [261]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [262]:
def get_pop_iris(year):
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

In [263]:
def get_iris_matrix(time: int, time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy() # Code à porter à l'appel
    return m



In [306]:
specialite=10
year=23
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, specialite, source, time, time_type, accessibilite_exp)

23 10 PS 30 HC -0.12


In [202]:
# Very long
iris_matrix = get_iris_matrix(time, time_type)
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,10010000,10010000,0,0,0,0,10010000
1,10010000,10050000,21,24,24,24,10050000
2,10010000,10210000,25,30,30,30,10210000
3,10010000,10250000,22,28,30,28,10250000
4,10010000,10260000,20,25,27,25,10260000
...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000
19272671,2023650000,2022510000,10,15,15,15,2022510000
19272672,2023650000,2022770000,16,28,28,28,2022770000
19272673,2023650000,2023650000,0,0,0,0,2023650000


In [307]:
pop_iris = get_pop_iris(year)
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,821300000,821300000,82130,C,170.000000,1.024096,3.072289,8.192771,5.120482,4.096386,28.674699,33.795181,26.626506,44.036145,15.361446
1,811540000,811540000,81154,C,80.000000,2.105263,3.157895,5.263158,4.210526,2.105263,4.210526,17.894737,17.894737,16.842105,6.315789
2,840060000,840060000,84006,C,71.000000,2.028571,3.042857,9.128571,3.042857,2.028571,16.228571,12.171429,9.128571,6.085714,8.114286
3,821570000,821570000,82157,C,107.000000,4.734513,3.787611,4.734513,5.681416,7.575221,16.097345,18.938053,17.991150,22.725664,4.734513
4,840070113,840070113,84007,A,2744.168077,66.531592,76.166839,91.027356,186.769463,466.091795,711.988855,459.891850,281.679718,253.076015,150.944594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48550,940460103,940460103,94046,A,1773.227226,43.209398,54.214737,66.914813,90.574673,141.873872,412.719594,339.034379,197.545292,327.448839,99.691628
48551,940460109,940460109,94046,A,2474.409868,60.984598,64.709729,183.003802,264.548488,197.623284,425.604137,527.371166,305.141054,330.464804,114.958805
48552,940460202,940460202,94046,A,2435.011871,90.000439,77.000375,125.000609,198.000965,427.002082,680.003315,670.003266,145.000707,17.000083,6.000029
48553,940540105,940540105,94054,A,2075.664303,59.233979,69.397063,104.184780,218.764070,149.651718,291.132829,414.269086,338.977538,337.581534,92.471705


In [308]:
ps_df = get_by_source(year, specialite, source)
ps_df

Quering PS for year 23 and specialite 10


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
0,647963,HASSAOUI,NADIA,1,185,13040000,5.334156,46.049236,L
1,647966,MARTIN,BRUNO,1,62992,13060000,4.884658,46.260609,L
2,647967,VIALARD,PHILIPPE,1,29,10330102,5.827150,46.108782,L
3,647972,PACI MENUT,ANNE,1,195,12480000,4.930047,45.895784,L
4,647973,BERNOU,HERVE,1,275334,10340102,5.686932,45.760931,L
...,...,...,...,...,...,...,...,...,...
53981,805361,BERNARD,SEVERINE,67,81024,672820000,7.499651,48.621024,L
53982,805364,BOUSSARD LEFEBVRE,ANNE,77,69924,771350000,2.960260,48.647657,L
53983,805365,MIGNON,BENJAMIN,77,292584,774490000,2.782739,48.843889,L
53984,805366,MIGNON,BENJAMIN,77,163956,773900202,2.650242,48.796171,L


In [309]:
nb_ps = ps_df["id"].nunique()
nb_cabinet_ps = ps_df.groupby(["id", "lon", "lat"])
print(f"Nb unique PS {nb_ps}")
print(f"Nb cabinet {len(nb_cabinet_ps)}")

Nb unique PS 50299
Nb cabinet 53364


In [310]:
ps_df["key"] = ps_df["id"].astype(str) + "_" + ps_df["code_mode_exercice"] + "_" + ps_df["lat"].astype(str) + "_" + ps_df["lon"].astype(str)
ps_df["nb_cabinet"] = ps_df.groupby("id")["key"].transform("nunique")
ps_df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.889416
2    0.099007
3    0.008817
4    0.002001
5    0.000278
7    0.000259
6    0.000222
Name: proportion, dtype: float64

In [311]:
ps_df[ps_df["id"]==1953]

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet


In [312]:
ps_df = ps_df[ps_df["code_mode_exercice"]=="L"]
ps_df["weight"] = 1 / ps_df["nb_cabinet"]
ps_df["nb"] = ps_df.groupby("iris")["weight"].transform("sum")
ps_df.head(10)
# 0.5 2.17
# 0.5 8.83

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,647963,HASSAOUI,NADIA,1,185,13040000,5.334156,46.049236,L,647963_L_46.04923648058139_5.334155951139914,1,1.0,8.5
1,647966,MARTIN,BRUNO,1,62992,13060000,4.884658,46.260609,L,647966_L_46.260609_4.884658,1,1.0,4.0
2,647967,VIALARD,PHILIPPE,1,29,10330102,5.827150,46.108782,L,647967_L_46.108782_5.82715,1,1.0,4.5
3,647972,PACI MENUT,ANNE,1,195,12480000,4.930047,45.895784,L,647972_L_45.895784_4.930047,1,1.0,2.0
4,647973,BERNOU,HERVE,1,275334,10340102,5.686932,45.760931,L,647973_L_45.760931_5.686932,1,1.0,10.0
5,647974,LE GUYADER,SOPHIE,1,155,10530401,5.227283,46.189234,L,647974_L_46.189234_5.227283,1,1.0,1.0
6,647976,BADEA,TATIANA,1,173,14260000,5.367903,46.270658,L,647976_L_46.270658_5.367903,1,1.0,2.0
7,647977,GUICHARD,CORALIE,1,78,14220000,5.315949,46.142116,L,647977_L_46.14211640222914_5.315948915942295,1,1.0,2.5
8,647978,VERMANDE,ARANZAZU,1,174,13760000,4.976799,45.829256,L,647978_L_45.829256_4.976799,1,1.0,3.0
9,647979,FONTES,AURELIE,1,275335,13030000,5.407107,46.086282,L,647979_L_46.0862816343406_5.40710681773977,1,1.0,4.0


In [313]:
ps_df = ps_df.sort_values(by='iris')
ps_df2 = ps_df.drop_duplicates(subset=['iris', 'nb'])
ps_df2


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
36801,750271,NEBOIT,VINCENT,1,274,10040101,5.342386,45.964146,L,750271_L_45.964146_5.342386,1,1.0,2.0
217,648324,PAGET,PHILIPPE,1,125,10040102,5.343799,45.954845,L,648324_L_45.95484508921275_5.343798989531327,1,1.0,2.0
42211,764864,SCHODTS,PIETER JAN,1,88441,10040201,5.356510,45.961711,L,764864_L_45.961711_5.35651,1,1.0,1.0
44948,772331,CHAVANT,JEREMY,1,259,10040202,5.350479,45.980772,L,772331_L_45.9807715_5.350478945319992,1,1.0,17.5
36800,750269,PIGNARD,DOMINIQUE,1,79597,10050000,4.902674,45.998332,L,750269_L_45.998332_4.902674,1,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47861,782002,ROUBELAT,SOLENNE,202,16600,2023350000,9.499933,42.332720,L,782002_L_42.33272_9.499933,1,1.0,2.0
4147,659393,MARTINETTI,ALEXANDRE,202,271158,2023420000,9.386235,41.920455,L,659393_L_41.920455_9.386235,1,1.0,1.0
4071,659261,CORAZZI,JEAN NOEL,202,16616,2023460000,9.464976,42.500392,L,659261_L_42.500392_9.464976,1,1.0,7.0
51074,788742,TERRIER,PHILIPPE,202,297565,2023540000,9.152516,42.152309,L,788742_L_42.152309_9.152516,2,0.5,0.5


In [314]:
iris_matrix_pop_df = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
iris_matrix_pop_df

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.0,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.0,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.0,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.0,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000,2B1350000,2B135,C,324.0,3.980971,2.049514,2.049514,2.020847,18.774109,42.392738,42.077099,55.572529,98.952018,56.130660
19272671,2023650000,2022510000,10,15,15,15,2022510000,2B2510000,2B251,C,3737.0,107.555612,118.720070,241.423529,317.512086,210.704067,631.501489,774.648158,501.913516,523.962099,309.059373
19272672,2023650000,2022770000,16,28,28,28,2022770000,2B2770000,2B277,C,333.0,4.577269,2.653059,12.288707,23.776840,23.014588,32.395245,76.114846,64.660108,67.979158,25.540181
19272673,2023650000,2023650000,0,0,0,0,2023650000,2B3650000,2B365,C,100.0,0.000000,0.943396,0.000000,3.773585,9.433962,9.433962,9.433962,23.584906,29.245283,14.150943


In [315]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)
iris_matrix_pop_df["accessibilite_weight"] = accessibilite_fn(iris_matrix_pop_df["time"])
iris_matrix_pop_df.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.000000,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.000000,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108,0.056135
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.000000,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055,0.027324
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.000000,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300,0.034735
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.000000,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472,0.049787
5,10010000,10280000,8,9,9,9,10280000,010280000,01028,C,621.000000,22.723364,29.574898,53.956390,70.586718,22.773615,107.898596,115.308426,89.365644,83.584630,25.227720,0.339596
6,10010000,10420000,14,16,16,16,10420000,010420000,01042,C,290.000000,3.133037,7.312245,22.902721,37.112788,10.199399,41.375370,72.152604,45.323311,42.956047,7.532476,0.146607
7,10010000,10460000,12,13,13,13,10460000,010460000,01046,C,904.000000,36.261938,27.238169,71.537021,113.956388,45.490399,158.230805,197.653434,122.489017,101.564554,29.578275,0.210136
8,10010000,10520000,17,21,22,21,10520000,010520000,01052,C,327.000000,8.933497,12.331319,22.701646,25.430244,8.343897,43.396597,63.486997,59.830525,57.972826,24.572452,0.080460
9,10010000,10530102,26,28,34,28,10530102,010530102,01053,A,1925.526865,47.250779,43.545566,63.504863,104.972858,183.815536,325.024244,323.908369,162.260767,385.334404,285.909479,0.034735


In [316]:
cols = [col for col in iris_matrix_pop_df.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [317]:
yy=min(year, 24)
sql = f"""
select o.* from apl.overrepresentation o
join specialite s on s.psp_spe_snds=o.psp_spe_snds
where o.year={yy}
and s.id={specialite}
"""
over = pd.read_sql(sql, config.connection_string)
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,23,1,0.976582,0.451643,0.336768,0.352023,0.640443,0.811534,0.874099,1.108385,1.624337,2.63959


In [318]:
if len(over) > 0:
    weights = over.values[0,2:]
else:
    weights = np.ones(len(over.columns) - 2)
weights

array([0.9765819 , 0.45164328, 0.33676791, 0.35202288, 0.64044287,
       0.81153415, 0.87409873, 1.1083846 , 1.6243372 , 2.63958955])

In [319]:
iris_matrix_pop_df["pop_gp"] = sum(w * iris_matrix_pop_df[c] for w, c in zip(weights, cols))
if specialite == 5:
    iris_matrix_pop_df["pop_gp"] /= 2
iris_matrix_pop_df=iris_matrix_pop_df.sort_values(by='iris2')
iris_matrix_pop_df.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000,828.207499
6351493,712700102,10010000,25,27,33,27,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.039164,828.207499
6351706,712700103,10010000,24,26,32,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,828.207499
60021,14430000,10010000,21,26,27,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,828.207499
6351935,712700104,10010000,25,28,34,28,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.034735,828.207499


In [320]:
if specialite!=5:
    test_pop=iris_matrix_pop_df.drop_duplicates(subset=['iris2'])
    ratio=test_pop["pop_gp"]/(test_pop["pop"]+1)
    ratio_mean=np.mean(ratio)
    print(ratio_mean)
    iris_matrix_pop_df["pop_gp"]=iris_matrix_pop_df["pop_gp"]/ratio_mean

1.0192276175210808


In [321]:
matrix_df = iris_matrix_pop_df[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
matrix_df.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C
6351493,712700102,10010000,25,27,0.039164,812.583455,832.0,C
6351706,712700103,10010000,24,26,0.044157,812.583455,832.0,C
60021,14430000,10010000,21,26,0.044157,812.583455,832.0,C
6351935,712700104,10010000,25,28,0.034735,812.583455,832.0,C


In [322]:
matrix_df["iris"] = matrix_df["iris1"].copy()
matrix_df = matrix_df.sort_values(by='iris2')
matrix_merge_df = matrix_df.merge(ps_df2, on="iris", how="left", suffixes=('', ''))
matrix_merge_df

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10280000,10010000,8,9,0.339596,812.583455,832.0,C,10280000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12890101,10010000,27,27,0.039164,812.583455,832.0,C,12890101,724117.0,SABATER,MARTINE,1.0,62996.0,5.203612,46.178872,L,724117_L_46.178872_5.203612,1.0,1.0,3.0
3,12890102,10010000,28,28,0.034735,812.583455,832.0,C,12890102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12910000,10010000,18,23,0.063292,812.583455,832.0,C,12910000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2023650000,0,0,1.000000,132.155571,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272671,2021350000,2023650000,6,8,0.382893,132.155571,100.0,C,2021350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272672,2022510000,2023650000,10,15,0.165299,132.155571,100.0,C,2022510000,765383.0,ROCCHI,ANDRE,202.0,16608.0,9.325426,42.010329,L,765383_L_42.01032905529371_9.325426327387799,1.0,1.0,5.0
19272673,2021230000,2023650000,17,25,0.049787,132.155571,100.0,C,2021230000,784387.0,BOCHU,LUCILE,202.0,16618.0,9.404639,42.015244,L,784387_L_42.015244_9.404639,1.0,1.0,3.0


In [323]:
matrix_merge_df["nb"] = matrix_merge_df["nb"].fillna(0)
matrix_merge_df = matrix_merge_df.sort_values(by=['iris1', "iris2"])
matrix_merge_df["nb"].nunique()

171

In [324]:
# matrix_merge_df = matrix_merge_df.sort_values(by='iris1')
matrix_merge_df["wpop"] = matrix_merge_df["accessibilite_weight"] * matrix_merge_df["pop_gp"]
matrix_merge_df["swpop"] = matrix_merge_df.groupby("iris1")["wpop"].transform("sum")
matrix_merge_df["R"] = matrix_merge_df["nb"] / (matrix_merge_df["swpop"] / 100000)
matrix_merge_df.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0
917,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0
2030,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0
2594,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0
2739,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0


In [325]:
rgp = matrix_merge_df[matrix_merge_df["iris1"] == matrix_merge_df["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
0,10010000,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.000000
177,10020000,10020000,C,249.082431,0.0,0.000000,8927.820518,249.082431,249.082431,267.000000
265,10040101,10040101,A,1667.757499,2.0,6.145209,32545.680096,1667.757499,1667.757499,1796.951774
470,10040102,10040102,A,3626.291232,2.0,6.595003,30325.988937,3626.291232,3626.291232,3837.421806
585,10040201,10040201,A,4278.192273,1.0,3.280914,30479.311660,4278.192273,4278.192273,4393.228271
...,...,...,...,...,...,...,...,...,...,...
19272636,2023560000,2023560000,C,199.692830,0.0,0.000000,244.778550,199.692830,199.692830,148.000000
19272651,2023610000,2023610000,C,308.556888,0.0,0.000000,1887.989599,308.556888,308.556888,306.000000
19272667,2023640000,2023640000,C,51.187847,0.0,0.000000,160.421017,51.187847,51.187847,36.000000
19272670,2023650000,2023650000,C,132.155571,0.0,0.000000,1161.476181,132.155571,132.155571,100.000000


In [326]:
apl = matrix_merge_df.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.000000,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
1,10050000,10010000,21,24,0.056135,812.583455,832.0,C,10050000,750269.0,PIGNARD,DOMINIQUE,1.0,79597.0,4.902674,45.998332,L,750269_L_45.998332_4.902674,1.0,1.0,2.0,45.614180,38143.971797,5.243292,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
2,10210000,10010000,25,30,0.027324,812.583455,832.0,C,10210000,648143.0,AMIEUX,AURELIE,1.0,104.0,4.820810,45.993790,L,648143_L_45.99379_4.82081,1.0,1.0,4.0,22.202805,46443.947918,8.612532,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
3,10250000,10010000,22,28,0.034735,812.583455,832.0,C,10250000,731830.0,ETIENNE,BENJAMIN,1.0,550.0,4.928185,46.312270,L,731830_L_46.312270023745924_4.928185320663883,1.0,1.0,1.0,28.225297,26201.721810,3.816543,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
4,10260000,10010000,20,25,0.049787,812.583455,832.0,C,10260000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,40.456148,34193.561547,0.000000,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2022730000,2022970000,3,7,0.431711,54.715414,53.0,C,2022730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,23.621220,349.438133,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272671,2022970000,2022970000,0,0,1.000000,54.715414,53.0,C,2022970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.715414,150.136226,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272672,2022990000,2022970000,6,11,0.267135,54.715414,53.0,C,2022990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.616419,692.901504,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272673,2023440000,2022970000,11,23,0.063292,54.715414,53.0,C,2023440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.463035,625.746806,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0


In [327]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.0,0.000000
155,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0,10050000,C,1777.796978,2.0,5.243292,38143.971797,1777.796978,1777.796978,1897.0,0.294331
386,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0,10210000,C,1367.765255,4.0,8.612532,46443.947918,1367.765255,1367.765255,1498.0,0.235326
653,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0,10250000,C,3735.163232,1.0,3.816543,26201.721810,3735.163232,3735.163232,4065.0,0.132569
808,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0,10260000,C,1047.065131,0.0,0.000000,34193.561547,1047.065131,1047.065131,979.0,0.000000


In [328]:
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.0,0.000000,34.159146
155,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0,10050000,C,1777.796978,2.0,5.243292,38143.971797,1777.796978,1777.796978,1897.0,0.294331,34.159146
386,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0,10210000,C,1367.765255,4.0,8.612532,46443.947918,1367.765255,1367.765255,1498.0,0.235326,34.159146
653,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0,10250000,C,3735.163232,1.0,3.816543,26201.721810,3735.163232,3735.163232,4065.0,0.132569,34.159146
808,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0,10260000,C,1047.065131,0.0,0.000000,34193.561547,1047.065131,1047.065131,979.0,0.000000,34.159146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272636,2023650000,2021350000,6,8,0.382893,451.234348,324.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,172.774422,1161.476181,0.0,2021350000,C,451.234348,0.0,0.000000,1801.097763,451.234348,451.234348,324.0,0.000000,17.822241
19272607,2023650000,2022510000,10,15,0.165299,3825.910203,3737.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,632.418703,1161.476181,0.0,2022510000,C,3825.910203,5.0,93.055898,5373.114578,3825.910203,3825.910203,3737.0,15.382036,17.822241
19272612,2023650000,2022770000,16,28,0.034735,368.164035,333.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.788273,1161.476181,0.0,2022770000,C,368.164035,0.0,0.000000,1437.844031,368.164035,368.164035,333.0,0.000000,17.822241
19272620,2023650000,2023650000,0,0,1.000000,132.155571,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,132.155571,1161.476181,0.0,2023650000,C,132.155571,0.0,0.000000,1161.476181,132.155571,132.155571,100.0,0.000000,17.822241


In [329]:
apl2 = apl[apl["iris1"] == apl["iris2"]]
apl2

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.0,812.583455,832.000000,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.000000,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.000000,0.000000,34.159146
28748,10020000,10020000,0,0,1.0,249.082431,267.000000,C,10020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,249.082431,8927.820518,0.000000,10020000,C,249.082431,0.0,0.000000,8927.820518,249.082431,249.082431,267.000000,0.000000,37.730438
28815,10040101,10040101,0,0,1.0,1667.757499,1796.951774,A,10040101,750271.0,NEBOIT,VINCENT,1.0,274.0,5.342386,45.964146,L,750271_L_45.964146_5.342386,1.0,1.0,2.0,1667.757499,32545.680096,6.145209,10040101,A,1667.757499,2.0,6.145209,32545.680096,1667.757499,1667.757499,1796.951774,6.145209,112.353943
28999,10040102,10040102,0,0,1.0,3626.291232,3837.421806,A,10040102,648324.0,PAGET,PHILIPPE,1.0,125.0,5.343799,45.954845,L,648324_L_45.95484508921275_5.343798989531327,1.0,1.0,2.0,3626.291232,30325.988937,6.595003,10040102,A,3626.291232,2.0,6.595003,30325.988937,3626.291232,3626.291232,3837.421806,6.595003,108.507336
29143,10040201,10040201,0,0,1.0,4278.192273,4393.228271,A,10040201,764864.0,SCHODTS,PIETER JAN,1.0,88441.0,5.356510,45.961711,L,764864_L_45.961711_5.35651,1.0,1.0,1.0,4278.192273,30479.311660,3.280914,10040201,A,4278.192273,1.0,3.280914,30479.311660,4278.192273,4278.192273,4393.228271,3.280914,117.057485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19268570,2023560000,2023560000,0,0,1.0,199.692830,148.000000,C,2023560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,199.692830,244.778550,0.000000,2023560000,C,199.692830,0.0,0.000000,244.778550,199.692830,199.692830,148.000000,0.000000,0.000000
19268441,2023610000,2023610000,0,0,1.0,308.556888,306.000000,C,2023610000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,308.556888,1887.989599,0.000000,2023610000,C,308.556888,0.0,0.000000,1887.989599,308.556888,308.556888,306.000000,0.000000,24.936006
19268523,2023640000,2023640000,0,0,1.0,51.187847,36.000000,C,2023640000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,51.187847,160.421017,0.000000,2023640000,C,51.187847,0.0,0.000000,160.421017,51.187847,51.187847,36.000000,0.000000,0.000000
19272620,2023650000,2023650000,0,0,1.0,132.155571,100.000000,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,132.155571,1161.476181,0.000000,2023650000,C,132.155571,0.0,0.000000,1161.476181,132.155571,132.155571,100.000000,0.000000,17.822241


In [330]:
print(year, specialite, source)
apl2["apl"].describe()
# 21-10-PA:67-57 avec ratio:68-58
# 20-10-PA:68-58 avec ratio:69-59

23 10 PS


count    48569.000000
mean        62.961648
std         33.189402
min          0.000000
25%         39.537216
50%         55.293727
75%         79.948876
max        484.349717
Name: apl, dtype: float64

In [296]:
# 21 10 PA
# count    48569.000000
# mean        66.239008
# std         36.200523
# min          0.000000
# 25%         40.432111
# 50%         56.872725
# 75%         85.083597
# max        681.212229

# 23 10 PA
# count    48569.000000
# mean        67.704332
# std         37.706173
# min          0.000000
# 25%         40.971273
# 50%         57.986870
# 75%         86.747319
# max        671.098376

# Avec prise en compte des cabinets non L
# 23 10 PA
# count    48569.000000
# mean        64.104830
# std         35.929837
# min          0.000000
# 25%         38.716316
# 50%         55.025585
# 75%         82.063870
# max        671.098376

In [297]:
dico = {"year":year, "specialite_id":specialite, "source":source, "time":time, "time_type":time_type, "exp":accessibilite_exp}
dico["mean"] = np.mean(apl2["apl"])
dico["std"] = np.std(apl2["apl"])
dico["q10"], dico["q25"], dico["q50"], dico["q75"], dico["q90"] = np.quantile(apl2["apl"], [0.1, 0.25, 0.5, 0.75, 0.9])
dico["min"] = np.min(apl2["apl"])
dico["max"] = np.max(apl2["apl"])
dico["date"] = datetime.datetime.now()
dico["key"] = random.randint(0, 1000000000000)
dico

{'year': 23,
 'specialite_id': 10,
 'source': 'PA',
 'time': 30,
 'time_type': 'HC',
 'exp': -0.12,
 'mean': 64.10482958844047,
 'std': 35.929467336132795,
 'q10': 27.56884058546753,
 'q25': 38.71631624897767,
 'q50': 55.02558476239584,
 'q75': 82.06386975269842,
 'q90': 115.99903198802471,
 'min': 0.0,
 'max': 671.0983762804187,
 'date': datetime.datetime(2025, 11, 4, 12, 21, 54, 792071),
 'key': 1103194203}

In [298]:
study = pd.DataFrame(dico, index=[dico["key"]])
study

,year,specialite_id,source,time,time_type,exp,mean,std,q10,q25,q50,q75,q90,min,max,date,key
1103194203,23,10,PA,30,HC,-0.12,64.10483,35.929467,27.568841,38.716316,55.025585,82.06387,115.999032,0.0,671.098376,2025-11-04 12:21:54.792071,1103194203


In [299]:
apl2[apl2["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1036747,50030000,50030000,0,0,1.0,439.665319,377.0,C,50030000,573093.0,BOREL ATGER,FLORENCE,5.0,273478.0,6.870576,44.783525,L,573093_L_44.783525_6.870576,3.0,0.333333,5.5,439.665319,920.235441,597.673134,50030000,C,439.665319,5.5,597.673134,920.235441,439.665319,439.665319,377.0,597.673134,597.673134
1215278,72620000,72620000,0,0,1.0,208.981651,180.0,C,72620000,240199.0,HEIJERMANS,HERMAN,7.0,1195596.0,3.970603,44.606850,L,240199_L_44.60685_3.970603,1.0,1.000000,2.0,208.981651,298.018900,671.098376,72620000,C,208.981651,2.0,671.098376,298.018900,208.981651,208.981651,180.0,671.098376,671.098376


In [300]:
apl3 = apl2[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl3 = apl3.rename(columns={'iris1': 'iris'})
apl3.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
0,10010000,C,0.00,32.678710,0.000000,1.0,812.583455,19422.017054,0.000000,812.583455,832.000000
28748,10020000,C,0.00,37.691066,0.000000,1.0,249.082431,8927.820518,0.000000,249.082431,267.000000
28815,10040101,A,0.00,105.037626,0.000000,1.0,1667.757499,32545.680096,0.000000,1667.757499,1796.951774
28999,10040102,A,2.00,105.165763,6.595003,1.0,3626.291232,30325.988937,6.595003,3626.291232,3837.421806
29143,10040201,A,7.25,112.203354,23.786626,1.0,4278.192273,30479.311660,23.786626,4278.192273,4393.228271


In [301]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2023 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id



,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,821300000,821300000,Montjoi,82,82130,Montjoi,2023,10
1,811540000,811540000,Marnaves,81,81154,Marnaves,2023,10
2,840060000,840060000,Auribeau,84,84006,Auribeau,2023,10
3,821570000,821570000,Saint-Beauzeil,82,82157,Saint-Beauzeil,2023,10
4,840070113,840070113,Carmes-Infirmières-Thiers,84,84007,Avignon,2023,10
...,...,...,...,...,...,...,...,...
48564,940460103,940460103,Centre Vert de Maison 3,94,94046,Maisons-Alfort,2023,10
48565,940460109,940460109,Centre Vert de Maison 9,94,94046,Maisons-Alfort,2023,10
48566,940460202,940460202,Alfort 2,94,94046,Maisons-Alfort,2023,10
48567,940540105,940540105,Carrières-Sentiers,94,94054,Orly,2023,10


In [302]:
apl_final = apl3.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl_final["year"]=year
apl_final

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,23,10,10010000,010010000,C,0.00,32.678710,0.000000,1.0,0.000000,812.583455,19422.017054,812.583455,832.000000,L'Abergement-Clémenciat,1,01001,L'Abergement-Clémenciat
1,23,10,10020000,010020000,C,0.00,37.691066,0.000000,1.0,0.000000,249.082431,8927.820518,249.082431,267.000000,L'Abergement-de-Varey,1,01002,L'Abergement-de-Varey
2,23,10,10040101,010040101,A,0.00,105.037626,0.000000,1.0,0.000000,1667.757499,32545.680096,1667.757499,1796.951774,Les Pérouses-Triangle d'Activités,1,01004,Ambérieu-en-Bugey
3,23,10,10040102,010040102,A,2.00,105.165763,6.595003,1.0,6.595003,3626.291232,30325.988937,3626.291232,3837.421806,Longeray-Gare,1,01004,Ambérieu-en-Bugey
4,23,10,10040201,010040201,A,7.25,112.203354,23.786626,1.0,23.786626,4278.192273,30479.311660,4278.192273,4393.228271,Centre-Saint-Germain-Vareilles,1,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,23,10,2023560000,2B3560000,C,0.00,0.000000,0.000000,1.0,0.000000,199.692830,244.778550,199.692830,148.000000,Zalana,202,2B356,Zalana
48565,23,10,2023610000,2B3610000,C,0.00,25.037451,0.000000,1.0,0.000000,308.556888,1887.989599,308.556888,306.000000,Zilia,202,2B361,Zilia
48566,23,10,2023640000,2B3640000,C,0.00,0.000000,0.000000,1.0,0.000000,51.187847,160.421017,51.187847,36.000000,Zuani,202,2B364,Zuani
48567,23,10,2023650000,2B3650000,C,0.00,16.284038,0.000000,1.0,0.000000,132.155571,1161.476181,132.155571,100.000000,San-Gavino-di-Fiumorbo,202,2B365,San-Gavino-di-Fiumorbo


In [303]:
d6=apl_final[apl_final["dept"]==6]
d6

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2095,23,10,60010000,060010000,C,0.000000,24.562258,0.000000,1.0,0.000000,97.746664,297.692517,97.746664,93.000000,Aiglun,6,06001,Aiglun
2096,23,10,60020000,060020000,C,0.000000,0.000000,0.000000,1.0,0.000000,60.236610,276.294969,60.236610,51.000000,Amirat,6,06002,Amirat
2097,23,10,60030000,060030000,C,0.000000,26.742784,0.000000,1.0,0.000000,606.375917,1311.798003,606.375917,640.000000,Andon,6,06003,Andon
2098,23,10,60040101,060040101,C,0.000000,143.700324,0.000000,1.0,0.000000,1646.292572,187948.770973,1646.292572,1763.364267,Les Trois Moulins,6,06004,Antibes
2099,23,10,60040102,060040102,A,3.000000,114.574502,2.000711,1.0,2.000711,2458.248090,149946.721270,2458.248090,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,23,10,61610105,061610105,A,8.700000,148.318824,4.312566,1.0,4.312566,3929.144712,201736.053993,3929.144712,3584.453834,Marina,6,06161,Villeneuve-Loubet
2591,23,10,61610106,061610106,A,0.000000,108.652539,0.000000,1.0,0.000000,3764.565930,146204.453272,3764.565930,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
2592,23,10,61610107,061610107,A,0.333333,118.347538,0.208379,1.0,0.208379,2691.559968,159964.686587,2691.559968,2766.654836,Plan,6,06161,Villeneuve-Loubet
2593,23,10,61620000,061620000,C,0.083333,21.715271,4.102075,1.0,4.102075,853.716514,2031.492367,853.716514,719.000000,La Brigue,6,06162,La Brigue


In [304]:
d6["apl"].describe()
# Ca change à cause des cabinets multi-dept

count    500.000000
mean      94.644163
std       49.025266
min        0.000000
25%       63.372694
50%      102.237839
75%      133.659490
max      308.069517
Name: apl, dtype: float64

In [41]:
# apl_final.to_csv("apl_france.csv", index=False)

In [42]:
# apl_final["study_key"]=dico["key"]
# study.to_sql("apl_study", config.connection_string, schema="apl", if_exists="append", index=False)
# apl_final.to_sql("apl", config.connection_string, schema="apl", if_exists="append", index=False)